# Notebook to calculate Inter Annotator Agreement

### Import the libraries

In [6]:
import sys
import pandas as pd
import collections 
import os
import numpy as np
from itertools import chain
from itertools import combinations
#sys.path.append('/home/jkuettel/NLP_spark/src')
#sys.path.append('/home/jkuettel/NLP_spark')
sys.path.append('/Users/l.kaack/Documents/Policy_coding/NLP_Spark/src')
sys.path.append('/Users/l.kaack/Documents/Policy_coding/NLP_Spark/')
from src.experiment_utils.helper_classes import token, span, repository
from src.d02_corpus_statistics.corpus import Corpus
from src.d03_inter_annotator_agreement.inter_annotator_agremment import Inter_Annotator_Agreement, _get_score_article
from definitions import df_annotation_marker
from src.d03_inter_annotator_agreement.inter_annotator_agremment import row_to_span_list, keep_valid_anotations


from definitions import ROOT_DIR


FileNotFoundError: [Errno 2] No such file or directory: '/home/jkuettel/NLP_spark/src/experiment_utils/tag_set.json'

## Small Tutorial

Load the dataframe stat_df

In [3]:
    
dataframe_dir = os.path.join(ROOT_DIR,'data/02_processed_to_dataframe', 'preprocessed_dataframe.pkl')
stat_df = pd.read_pickle(dataframe_dir)
stat_df.head()

NameError: name 'ROOT_DIR' is not defined

First create a object of class Inter annotator agreement. The constructor takes a stat_df as input, has a optional argument DEBUG where only the first 10 articles are taken to test different functions

In [ ]:
test_evaluator = Inter_Annotator_Agreement(stat_df)
test_evaluator_debug = Inter_Annotator_Agreement(stat_df, DEBUG = True)

Inter_Annotator_Agreement is a child class of the Corpus class, so all methods of the Corpus class are available

In [ ]:
test_dir = repository(policy = 'EU_32008R1099')
test_evaluator.get_span_list(conditional_rep = test_dir, columns = 'annotators', item = 'tag', value =  'Tech_LowCarbon')

To calculate the inter annonator agreement, there are two options


## Append the score to dataframe

This method appends the inter-annotator agreement for each article which at least two valid annoations based on a set of inter-annotator agreement measures. The scores are calculated in parallel, this is the recommended method for computationally intensive scores.

First, we only consider the articles where the curation is finished and at least two annotators are present:

In [ ]:
test_evaluator.keep_only_finished_articles()

In [ ]:
scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial', 'f1_heuristic'] #, 'pygamma']

In [ ]:
test_evaluator.append_total_score_per_article(scoring_metrics)

There is also a normal implementation which uses parallel

In [ ]:
test_evaluator.append_total_score_per_article_parallel(scoring_metrics)

Checking out the dataframe now:

In [ ]:
test_evaluator.df.head()

### Get total score

To retrieve the total score of the corpus, use get_total_score_df() on a dataframes where the scores for individual articles have been calculated.

In [ ]:
test_evaluator.get_total_score_df()

if only specific scores are required:

In [ ]:
test_evaluator.get_total_score_df('f1_exact_score')

or

In [ ]:
test_evaluator.get_total_score_df(['f1_exact_score', 'f1_tokenwise_score'])

### Get total score per annotator

In [ ]:
annotators = ['Onerva', 'Alisha', 'Fabian', 'Fride']
for ann in annotators:
    print('annotator: ', ann)
    print(test_evaluator.get_total_score_df(annotator = ann)
    print('----------------')

In [ ]:
test_evaluator.get_score_annotator('Fride', ['f1_exact_score', 'f1_tokenwise_score'] )

### Rank articles by score

In [ ]:
test_evaluator.df.sort_values(by=['f1_heuristic_score'])

## Get total score based on a spanlist

The inter annotator agreement score can be also calculated from a spanlist. For all the spans present, it calculates the inter agreement scores for alle the articls with at least two valid annoations. Can be used to caluclate simmilarity to curation.

In [ ]:
test_dir = repository.from_repository_name('EU_32006L0032_Title_0_Chapter_1_Section_0_Article_03')
span_list = test_evaluator.get_span_list(test_dir, ['Onerva', 'Fride'])

In [ ]:
test_evaluator.get_score_spanlist(span_list, 'f1_heuristic')

## Check closeness to curation

To check the agreement with the curation of all annotators, we simply create a spanlist for each annotator containing all his spans and the ones from the curation
Since this is based on a big list instead of a dataframe, the computation is very slow

In [ ]:
annotators = ['Onerva', 'Alisha', 'Fabian', 'Fride']
repo = repository()

for ann in annotators:
    span_list_annotator_curation = test_evaluator.get_span_list(repo, ['Curation', ann])
    score = test_evaluator.get_score_spanlist(span_list_annotator_curation, 'f1_heuristic')
    print('annotator: ', ann, ', score: ', score)
    
    
    
    

# Check scores in different categories

In [ ]:
layers = ['Technologyandapplicationspecificity', 'Policydesigncharacteristics', 'Instrumenttypes' ]
repo = repository()

for l in layers:
    span_list_layer = test_evaluator.get_span_list(repo, columns = 'annotators', item = 'layer', value = l)
    score = test_evaluator.get_score_spanlist(span_list_layer, 'f1_heuristic')
    print('layer: ', l, ', score: ', score)
    

In [ ]:
all_tags = 